# Servier - List
## Beachten
* Gewisse Spalten sind mit Linie, gewisse nicht. Deshalb Spalten manuell definiert
* Hat Rundungsdifferenzen

In [4]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [5]:
# Read pdf into DataFrame
columns = [68.38, 203.1, 254.48, 275.92, 442.91, 548.04, 574.19, 606.36, 639.9, 665.59, 688.21, 732.45]
df = tabula.read_pdf("2017 SERVIER SUISSE SA DEUTSCH 28.6.18.pdf", pages='all', lattice=False, columns=columns, guess=False)

## Format Table

In [13]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['empty0', 'name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses', 'total']

#Copy rows with name but no value to the next row (was multiple row)
for index, row in df_export[df_export.name.notna() & df_export.total.isna()].iterrows():
    df_export.loc[index + 1, 'name'] = row['name'] + ' ' + str(df_export.loc[index + 1, 'name'])
    
#Add Type
index_hco = df_export[df_export['name'].str.contains("ANDERE, OBEN NICHT GENANNTE", na=False)].index[1]
df_export['type'] = np.where(df_export.index < index_hco, 'hcp', 'hco')

#Remove empty rows
df_export = df_export.dropna(subset=['address'], how='all')
df_export = df_export.dropna(subset=['total'], how='all')

#Remove empties
df_export.drop(columns=['empty0'], inplace=True)

#Remove Titles
df_export['name'] = df_export.name.str.replace(regex_title, '', regex=True, case=False)

#Cleanup Numbers and sum
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)

#Export PLZ
df_export['plz'] = df_export['location'].str.extract(r'(^\S*)', expand=True)

#basic string conversion
df_export = basic_string_conversion(df_export)

export_list(df_export, 'servier')

Total nicht Summe der Werte
saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx')